In [1]:
from chords_and_notes_dict import * 
import threading 
import tkinter as tk
from PIL import ImageTk, Image
import mido
import random 
import os
from functools import partial
import pickle
import time
from datetime import datetime
import copy
import rtmidi as rt

# Script

In [ ]:
current_path = os.getcwd()
green_keys_path = os.path.join(current_path, "green_keys")
red_keys_path = os.path.join(current_path, "red_keys")

In [2]:
def open_port(input_name: str, output_name: str): 
    """ 
    Input_port: MIDI controller to send MIDI messages 
    Output_port: Corresponds to the sound that the programm returns after getting MIDI message --> it can be no sound 

    Case 1: If input and output ports are available --> open them with the names given in parameters
    Case 2: If input and output ports are already used --> close them and open them with the names given in parameters

    It returns the input_port and output_port 
    """

    global input_port, output_port

    #if input_port and output_port are AVAILABLE
    try: 
        input_port = mido.open_input(input_name)
        output_port = mido.open_output(output_name)

    #if input_port and output_port are NOT AVAILABLE    
    except Exception:
        close_port(input_port, output_port)
        input_port = mido.open_input(input_name)
        output_port = mido.open_output(output_name)
    
    return input_port, output_port 


def close_port(input_port, output_port) -> None: 
    """ 
    Close input and output ports
    """
    input_port.close()
    output_port.close()


def choose_ports():
    """ 
    Create a Tkinter window in which the input and output names are displayed on your machine
    
    - Select ONLY one for the input by clicking inside a checkbox in the INPUTS column
    - Select ONLY one for the ouput by clicking inside a checkbox in the OUTPUTS column

    It returns the input_name and output_name selected
    """

    def validate_checkboxes():
        """ 
        Retrieve the names of the input and output selected by clicking inside their checkbox
        Then it kills the Tkinter "Inputs & Outputs" window
        """

        global selected_input, selected_output

        input_index = [var.get() for var in input_vars].index(1)
        selected_input = input_options[input_index]
        output_index = [var.get() for var in output_vars].index(1)
        selected_output = output_options[output_index]

        print(f"Input selected: '{selected_input}'")
        print(f"Output selected: '{selected_output}'")

        window.destroy()


    window = tk.Tk()
    window.geometry("600x400")

    input_label = tk.Label(window, text="INPUTS", font=("Aerial", 20))
    input_label.grid(row=0, column=0, sticky="w")

    output_label = tk.Label(window, text="OUTPUTS", font=("Aerial", 20))
    output_label.grid(row=0, column=1, sticky="w")

    # Create the lists of input and output options available on your machine
    input_options = mido.get_input_names()
    output_options = mido.get_output_names()

    # Create variables to track the state of the checkboxes
    input_vars = []
    output_vars = []

    # Create checkboxes for the input options
    for i, option in enumerate(input_options):
        var = tk.IntVar()
        checkbox = tk.Checkbutton(window, text=option, variable=var)
        checkbox.grid(row=i+1, column=0, sticky="w")
        input_vars.append(var)

    # Create checkboxes for the output options
    for i, option in enumerate(output_options):
        var = tk.IntVar()
        checkbox = tk.Checkbutton(window, text=option, variable=var)
        checkbox.grid(row=i+1, column=1, sticky="w")
        output_vars.append(var)

    # Create the entry button to validate the checkboxes
    button = tk.Button(window, text="Validate", command=validate_checkboxes)
    button.grid(row=max(len(input_options), len(output_options))+1, columnspan=2)

    window.mainloop()

    return selected_input, selected_output




# Phases de test

In [ ]:
def process_midi_messages(input_port):
        """ 
        Process the MIDI message function when a key on the controller is pressed("note_on") and unpressed ("note_off")
        This function is the threaded function
        """

        global note_container, msg
        note_container = []
        
        
        for msg in input_port:
            if msg.type == "note_on" or msg.type == "note_off": 
                if not fonction_break: #if True, run the threaded function / if False, don't run this threaded function 
                    handle_midi_message(msg, output_port)
            else: 
                pass


def start_midi_processing(input_port):
    """
    Enable the function "process_midi_messages" to be threaded
    As this function contains a "for loop" of midi messages input, it is endless. But we need to access also to the "root.mainloop()" event. So we thread this function    
    """
    midi_thread = threading.Thread(target=process_midi_messages, args=(input_port,))
    midi_thread.start()

In [ ]:
def check_answer_note_request_mode(function_called):
    """ 
    - Check for the request_note mode, if the note value (from 0 to 127) of the key pressed is in the list of the good answers (list_notes_true_answer)
    - Save in the corresponding dictionaries (good/bad answer and time) the name of the request
    - Destroy the label of the note request widget to release memory space for next qureries
    - Automatically invoke another call of the activate_function aka "request_note" function for another note interval request
    """

    global note_path, start_time_note

    good_answer_key = list(dict_to_save.keys())[0]
    bad_answer_key = list(dict_to_save.keys())[1]
    good_time_key = list(dict_to_save.keys())[2]
    bad_time_key = list(dict_to_save.keys())[3]

    if msg.type == "note_on":
        #if the match is correct, select green_keys folder
        if note_value in list_notes_true_answer: 
            dict_to_save[good_answer_key].append(random_note_request)
            dict_to_save[good_time_key].append(time_response_note)

            #Choose folder of green notes
            note_path = green_keys_path                            

        #if the match is uncorrect, select red_keys folder
        else: 
            dict_to_save[bad_answer_key].append(random_note_request)
            dict_to_save[bad_time_key].append(time_response_note)

            #Choose folder of red notes
            note_path = red_keys_path

        #Delete overlay_label_note --> note_request
        overlay_label_note_request.destroy()

        #Call the activate_function (should be "request_note") automatically for another request
        activate_function()



def note_display(canvas, note_value: int):
    """ 
    Display the overlayed red or green notes over the piano background image when a key is pressed on MIDI controller and remove them when keys are unpressed 
    """

    # dynamic_variable = note_value # dynamic_variable is named as a variable whose name depends on the value of note_value. Thus it is a dynamic variable
    overlayed_widget = tk.Label(root) # As tk.Label can be only used for 1 image (or text), we assign this widget to dynamic_variable as we want simultaneous overlayed images on the piano background image

    #The display will depend on the exercise choser ("request a note, write a note or request chords")
    check_answer_note_request_mode(activate_function)

    #Depending on the position of the key we play on the MIDI controller (top black key or left, mid, right white key), open the corresponding overlayed image
    if NOTE_VALUES_DICT[note_value][1] == "upper":
        overlayed_image = Image.open(os.path.join(note_path, "upper_key.png"))
        overlayed_image = overlayed_image.resize((44, 230)) #resize to match top black key dimensions
    elif NOTE_VALUES_DICT[note_value][1] == "mid":
        overlayed_image = Image.open(os.path.join(note_path, "mid_key.png"))
        overlayed_image = overlayed_image.resize((60, 400)) #resize to match mid white key dimensions
    elif NOTE_VALUES_DICT[note_value][1] == "left":
        overlayed_image = Image.open(os.path.join(note_path, "left_key.png"))
        overlayed_image = overlayed_image.resize((60, 400)) #resize to match left white key dimensions
    elif NOTE_VALUES_DICT[note_value][1] == "right":
        overlayed_image = Image.open(os.path.join(note_path, "right_key.png"))
        overlayed_image = overlayed_image.resize((60, 400)) #resize to match right white key dimensions
        
    overlayed_image = ImageTk.PhotoImage(overlayed_image) #Create the overlayed image

    #Assign overlayed image to corresponding tk.Label
    overlayed_widget.config(image=overlayed_image)
    overlayed_widget.image = overlayed_image
    


def handle_midi_message(msg, output_port):
    """ 
    Receive the MIDI messages from the controller and play it, depending on "note_on" or "note_off" type 
    MIDI messages contain essentially: 
    - Type: ("note_on", "note_off")
    - Channel: the channel of MIDI messages (from 1 to 16)
    - Note: The pitch of the note played (from 0 to 127 where "60" is the middle C note and considered as the central reference point)
    - Velocity: Volume of the note (from 0 to 127) --> "note_off" corresponds to volcity=0

    Example of MIDI message: "note_on channel=0 note=52 velocity=63"
    """
    global note_value, start_time_note, time_response_note, end_time_note, note_path
    
    note_container.append(msg.note)
    output_port.send(msg) #send MIDI message through the specified MIDI output port (i.e. output_port in this case) --> make the sound if "note_on" and stop the sound if "note_off"
    note_value = msg.note

    if not display_key_pressed:
        pass

    elif msg.note >= 36 and msg.note <= 83: #display key pressed on background image 

        if activate_function is request_note:
            #Save time_response in variable and add to list later as we don't know as this step if the answer is correct or wrong
            end_time_note = time.time()
            time_response_note = "{:.4f}".format((end_time_note - start_time_note))

        elif activate_function is chords:
            if msg.type == "note_on":
                list_notes_stored.append(note_value)
                list_notes_stored.sort()
                if any(note_value in sublist for sublist in list_answers_chords):
                    note_path = green_keys_path
                else:
                    note_path = red_keys_path
            elif msg.type == "note_off" and len(list_notes_stored) != 0:
                print(list_notes_stored)
                print(note_value)
                list_notes_stored.remove(note_value)
            
            # print(list_notes_stored)

        #Display note on virtual piano
        note_display(canvas, note_value)
        
    else:
        if activate_function is chords:
            out_of_scale_label = tk.Label(root, text="ERROR !\nNote out of range, change octave", font=("Aerial", 20), fg="blue")
            out_of_scale_label.place(x=1000, y=460)
            root.after(1200, out_of_scale_label.destroy)
        pass #we can hear the sound but it is not displayed as the piano background image contains only notes between 36 and 83 included


def request_note() -> None:
    """ 
    Request a interval from a note. The user should answer by pressing the key on his MIDI controller they believe is the correct answer
    Example --> Request = "Minor seventh of G" --> User should press an F key on his piano keyboard

    Compute the response time starting from the call of this function 
    """

    global display_key_pressed, overlay_label_note_request, start_time_note, list_notes_true_answer, random_note_request, dict_to_save

    # Set the variable 'display_key_pressed' to True so that every time a note is pressed on the MIDI controller, the corresponding note on the virtual piano is highlighted
    display_key_pressed = True

    dict_to_save = load_dict_to_save(activate_function_for_save="request_note")

    #Pick random itnerval request from a base note
    random_note_request = random.choice(list(request_note_dict.keys()))

    #Select the corresponding true answer 
    list_notes_true_answer = NOTE_VALUE_MATCH(random_note_request)

    #Create the label where the note requests will be displayed
    overlay_label_note_request = tk.Label(root)
    overlay_label_note_request.place(x=650, y=450)
    overlay_label_note_request.config(text=random_note_request, font=("Arial", 20))

    #Start time calculation for response_time value
    start_time_note = time.time()



def button_clicked(button_name):
    """ 
    - Activate the game mode you have chosen 
    - Set quit_button_pressed to False indicating that you don't quit the game mode for now (for dict saving purpose)
    - Initialization of new buttons for sub modes in the game mode you have chosen 

    """

    global quit_button, next_button, activate_function, left_hand_button, right_hand_button, dict_to_save, quit_button_pressed, normal_button, reverse_button

    #Set quit_button to False, meaning that quit_button has not been pressed yet (for dict saving purpose)
    quit_button_pressed = False 

    activate_function = button_name
    
    #Delete temporary the 3 game modes buttons (still in the memory space)
    note_button.place_forget()
    write_button.place_forget()
    chords_button.place_forget()

    if activate_function is chords:
        #Create button for left hand exercise
        left_hand_button = tk.Button(root, text="LEFT HAND", height=5, width=15, command=partial(chords, hand_exercise="left_hand", activate_function_for_save="chords"))
        left_hand_button.place(x=550, y=450)

        #Create button for right hand exercise
        right_hand_button = tk.Button(root, text="RIGHT HAND", height=5, width=15, command=partial(chords, hand_exercise="right_hand", activate_function_for_save="chords"))
        right_hand_button.place(x=750, y=450)


    elif activate_function is write:
        #Create button for left hand exercise
        normal_button = tk.Button(root, text="NORMAL", height=5, width=15, command=partial(write, exercise_type="normal"))
        normal_button.place(x=550, y=450)

        #Create button for right hand exercise
        reverse_button = tk.Button(root, text="REVERSE", height=5, width=15, command=partial(write, exercise_type="reverse"))
        reverse_button.place(x=750, y=450)


    elif activate_function is request_note:
        quit_button = tk.Button(root, text="QUIT", height=3, width=12, command=quit)
        quit_button.place(x=350, y=500)

        next_button = tk.Button(root, text="NEXT", height=3, width=12, command=next)
        next_button.place(x=350, y=440)

        # Run the request_note function
        button_name() 


    # else: #write and note functions

    #     if activate_function is request_note:
    #         dict_to_save = load_dict_to_save(activate_function_for_save="request_note") 
    #JE NE SAIS PAS SI CES LIGNES SONT IMPORTANTES DONC JE LES GARDE POUR LE MOMENT



def load_dict_to_save(**kwargs):

    global dict_function

    if kwargs["activate_function_for_save"] == "request_note":
        dict_function = {"list_answer_good_note": [],
                              "list_answer_bad_note": [],
                              "list_time_good_note": [],
                              "list_time_bad_note": [],
                              }
    elif kwargs["activate_function_for_save"] == "write":
        if exercise_type == "normal":
            dict_function = {"list_answer_good_write": [],
                        "list_answer_bad_write": [],
                        "list_time_good_write": [],
                        "list_time_bad_write": [],
                        }
            
        elif exercise_type == "reverse":
            dict_function = {"list_answer_good_write_reverse": [],
                        "list_answer_bad_write_reverse": [],
                        "list_time_good_write_reverse": [],
                        "list_time_bad_write_reverse": [],
                        }
    
    elif kwargs["activate_function_for_save"] == "chords":
        if hand_exercise == "right_hand":
            dict_function = {"list_answer_good_chord_right": [],
                                 "list_answer_bad_chord_right": [],
                                 "list_time_good_chord_right": [],
                                 "list_time_bad_chord_right": [],
                                 }
        elif hand_exercise == "left_hand":
            dict_function = {"list_answer_good_chord_left": [],
                           "list_answer_bad_chord_left": [],
                           "list_time_good_chord_left": [],
                           "list_time_bad_chord_left": [],
                           }
            
    return dict_function

In [3]:





def initialize_buttons() -> None:
    """ 
    Initialize 3 buttons on the root tkinter window corresponding to the 3 game modes availbable: 
    - Note button: Play notes on the piano controller that have been requested 
    - Write button: Write the note with your keyboard that corresponds to the interval requested --> For example the request can be "Second major of C"
    - Chords button: Play the chords on the piano controller that have been requested

    Initialize the display_key_pressed variable to False, which indicates that while none of the game modes has been selected, you can play on your piano controller and get the 
    sound corresponding piano key but not the image of the piano key pressed on the piano background image 
    """
    
    global note_button, display_key_pressed, write_button, chords_button

    #Create a Button to request note
    note_button = tk.Button(root, text="NOTES", height=3, width=12, command=partial(button_clicked, request_note))
    note_button.place(x=0, y=410)

    #Create button that requests note appelation 
    write_button = tk.Button(root, text="WRITE", height=3, width=12, command=partial(button_clicked, write))
    write_button.place(x=0, y=470)

    #Create button that requests chords
    chords_button = tk.Button(root, text="CHORDS", height=3, width=12, command=partial(button_clicked, chords))
    chords_button.place(x=0, y=530)

    display_key_pressed = False #while the note_button is not pressed, we will just hear notes from controller, not the visual    



def main(width=1600, height=400) -> None:
    """ 
    1) Open the input and output ports 
    2) Display the virtual piano keyboard window with Tkinter    
    3) Run the initialize_buttons function which initialized the different "game" modes of the program --> Click on one of those to play the mode
    4) Run the threading function to receive/send MIDI messages while the background virtual piano is running 

    """

    global canvas, root, output_port, input_port

    #Open ouput and input port 
    input_chosen, output_chosen = choose_ports()
    input_port, output_port = open_port(input_chosen, output_chosen)
    
    # Initialize the Tkinter window 
    root = tk.Tk()
    root.geometry("1600x600")

    # Load the piano background image and resize it
    background_image = Image.open("Images\piano.png")
    background_image = background_image.resize((width, height))
    background_image = ImageTk.PhotoImage(background_image)  

    # Create the canvas with the background piano image inside the Tkinter window
    canvas = tk.Canvas(root, width=width, height=height)
    canvas.pack()
    canvas.create_image(0, 0, image=background_image, anchor=tk.NW)


    #Intialize buttons 
    initialize_buttons()

    #Thread the MIDI messages tasks
    start_midi_processing(input_port)   


    root.mainloop()